# ch 19_2 huggingface tokenizers

이번 챕터에서는 huggingface에서 제공하는 tokenizers 라이브러리를 이용해서 직접 corpus 데이터 셋을 가지고 subword tokenizer를 학습시켜 보겠습니다.

## 데이터 셋 준비

In [1]:
import pandas as pd

In [155]:
train_df = pd.read_csv("./data/naver_reviews_train.csv")
test_df = pd.read_csv("./data/naver_reviews_test.csv")

In [156]:
train_df.isnull().sum(), test_df.isnull().sum()

(document    5
 label       0
 dtype: int64,
 document    3
 label       0
 dtype: int64)

In [157]:
train_df = train_df.dropna()
test_df = test_df.dropna()

In [158]:
train_df.iat[0, 0] = "惡 최고의 영화"

In [178]:
# pattern = re.compile(r"[^ㄱ-힣a-zA-Z0-9!?., \']")
pattern = re.compile(r'[\u4e00-\u9fff]+')

In [172]:
sample = "惡 최고의 영화" 

In [173]:
pattern.sub("", sample)

' 최고의 영화'

In [179]:
import re

# 특수 문자 제거 및 소문자화
def preprocess_text(text: str) -> str:
    text = text.lower()
    text = pattern.sub("", text)
    # text = re.sub(r"[^ㄱ-힣a-zA-Z0-9!?.,\']+", r" ", text)
    text = re.sub(r"\s+", " ", text)
    return text

train_df["document"] = train_df["document"].apply(lambda x: preprocess_text(x))

In [180]:
train_df["document"].apply(lambda x: preprocess_text(x))

0                                                    최고의 영화
1                         흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나
2                                         너무재밓었다그래서보는것을추천한다
3                             교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정
4         사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...
                                ...                        
149995                                  인간이 문제지.. 소는 뭔죄인가..
149996                                        평점이 너무 낮아서...
149997                      이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?
149998                          청춘 영화의 최고봉.방황과 우울했던 날들의 자화상
149999                             한국 영화 최초로 수간하는 내용이 담긴 영화
Name: document, Length: 149995, dtype: object

In [181]:
train_df

,document,label
0,최고의 영화,0
1,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,너무재밓었다그래서보는것을추천한다,0
3,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
...,...,...
149995,인간이 문제지.. 소는 뭔죄인가..,0
149996,평점이 너무 낮아서...,1
149997,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0
149998,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1


## huggingface tokenizers

huggingface는 딥러닝을 이용한 NLP 

- transformers: 트랜스포머 기본 모델과 이를 응용한 NLP 모델들을 제공
- tokenizers: subword tokenizer 제공

In [30]:
!pip install tokenizers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [200]:
from tokenizers import Tokenizer
from tokenizers.models import WordPiece 
from tokenizers.trainers import WordPieceTrainer
from tokenizers.pre_tokenizers import Whitespace

trainer = WordPieceTrainer(
    vocab_size=10000,
    min_frequency=5,
    special_tokens=["[PAD]", "[UNK]", "[SOS]", "[EOS]"]
)

In [201]:
def batch_iterator(dataset, batch_size=1000):
    for i in range(0, len(dataset), batch_size):
        yield dataset[i: i+batch_size]

tokenizer = Tokenizer(WordPiece())
tokenizer.pre_tokenizer = Whitespace()
data = list(train_df["document"].values)
tokenizer.train_from_iterator(batch_iterator(data),  trainer=trainer)

In [202]:
len(data)

149995

In [203]:
for elem in data:
    if "惡" in elem:
        print(elem, "!!")
        break

In [204]:
vocabs = [(k, v) for k, v in tokenizer.get_vocab().items()]
sorted_vocabs = sorted(vocabs, key=lambda x: x[1])
print(sorted_vocabs)

[('[PAD]', 0), ('[UNK]', 1), ('[SOS]', 2), ('[EOS]', 3), ('!', 4), ("'", 5), (',', 6), ('.', 7), ('0', 8), ('1', 9), ('2', 10), ('3', 11), ('4', 12), ('5', 13), ('6', 14), ('7', 15), ('8', 16), ('9', 17), ('?', 18), ('a', 19), ('b', 20), ('c', 21), ('d', 22), ('e', 23), ('f', 24), ('g', 25), ('h', 26), ('i', 27), ('j', 28), ('k', 29), ('l', 30), ('m', 31), ('n', 32), ('o', 33), ('p', 34), ('q', 35), ('r', 36), ('s', 37), ('t', 38), ('u', 39), ('v', 40), ('w', 41), ('x', 42), ('y', 43), ('z', 44), ('ㄱ', 45), ('ㄲ', 46), ('ㄳ', 47), ('ㄴ', 48), ('ㄵ', 49), ('ㄶ', 50), ('ㄷ', 51), ('ㄸ', 52), ('ㄹ', 53), ('ㄺ', 54), ('ㄻ', 55), ('ㅀ', 56), ('ㅁ', 57), ('ㅂ', 58), ('ㅃ', 59), ('ㅄ', 60), ('ㅅ', 61), ('ㅆ', 62), ('ㅇ', 63), ('ㅈ', 64), ('ㅉ', 65), ('ㅊ', 66), ('ㅋ', 67), ('ㅌ', 68), ('ㅍ', 69), ('ㅎ', 70), ('ㅏ', 71), ('ㅐ', 72), ('ㅑ', 73), ('ㅒ', 74), ('ㅓ', 75), ('ㅔ', 76), ('ㅕ', 77), ('ㅖ', 78), ('ㅗ', 79), ('ㅘ', 80), ('ㅛ', 81), ('ㅜ', 82), ('ㅝ', 83), ('ㅞ', 84), ('ㅟ', 85), ('ㅠ', 86), ('ㅡ', 87), ('ㅢ', 88), ('ㅣ', 89), ('ㅤ

In [196]:
len(sorted_vocabs)

12000

## tokenizer 학습 여부 확인 및 저장

In [197]:
samples = [
    "너무 재밌어요, 꿀잼 인정!",
    "보다가 중간에 졸았습니다 ㅠㅠ 비추에요"
]

In [198]:
for sample in samples:
    output = tokenizer.encode(sample)
    print(output.ids)
    print(output.tokens)

[4903, 5418, 6, 6055, 6339, 4]
['너무', '재밌어요', ',', '꿀잼', '인정', '!']
[5312, 5867, 1856, 6991, 5063, 6236, 5408]
['보다가', '중간에', '졸', '##았습니다', 'ㅠㅠ', '비추', '##에요']


In [199]:
vocabs = tokenizer.get_vocab()


이때, 각 토큰의 앞에 ##이 붙은 것은 단어 안에 포함된 단어, subword를 가리키는 말입니다.